<a href="https://colab.research.google.com/github/markriis/IDS-pollution-of-cities/blob/main/Data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing data and librarys

In [73]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

!pip install -q opendatasets
import opendatasets as od


In [74]:
#air_and_water = pd.read_csv('https://raw.githubusercontent.com/markriis/IDS-pollution-of-cities/main/cities_air_quality_water_pollution2021.csv')

gasses = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv',
                     usecols=['country', 'year', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_change_co2_per_capita', 'methane',
       'methane_per_capita', 'nitrous_oxide', 'nitrous_oxide_per_capita',
       'oil_co2', 'oil_co2_per_capita', 'primary_energy_consumption',
       'total_ghg', 'total_ghg_excluding_lucf'])# Removed some columns that I doubt would be of use, meanings at https://github.com/owid/co2-data/blob/master/owid-co2-codebook.csv

air = pd.read_excel('https://cdn.who.int/media/docs/default-source/air-pollution-documents/air-quality-and-health/who_aap_2021_v9_11august2022.xlsx?sfvrsn=9035996c_3',
                    storage_options={'User-Agent': 'Mozilla/5.0'}, sheet_name='AAP_2022_city_v9',
                    usecols=['WHO Country Name', 'City or Locality', 'Measurement Year', 'PM2.5 (μg/m3)', 'PM10 (μg/m3)', 'NO2 (μg/m3)'])

development = pd.read_csv('https://hdr.undp.org/sites/default/files/2021-22_HDR/HDR21-22_Composite_indices_complete_time_series.csv',
                          storage_options={'User-Agent': 'Mozilla/5.0'},
                          usecols=['country', 'hdi_1990', 'hdi_1991', 'hdi_1992', 'hdi_1993',
       'hdi_1994', 'hdi_1995', 'hdi_1996', 'hdi_1997', 'hdi_1998',
       'hdi_1999', 'hdi_2000', 'hdi_2001', 'hdi_2002', 'hdi_2003',
       'hdi_2004', 'hdi_2005', 'hdi_2006', 'hdi_2007', 'hdi_2008',
       'hdi_2009', 'hdi_2010', 'hdi_2011', 'hdi_2012', 'hdi_2013',
       'hdi_2014', 'hdi_2015', 'hdi_2016', 'hdi_2017', 'hdi_2018',
       'hdi_2019', 'hdi_2020', 'hdi_2021'])#

dataset_url = 'https://public.opendatasoft.com/explore/dataset/geonames-all-cities-with-a-population-1000/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B'
od.download(dataset_url)# This one takes a while

populations = pd.read_csv('./geonames-all-cities-with-a-population-1000.csv', delimiter=';',
                          usecols=['Name', 'ASCII Name', 'Country name EN', 'Population', 'Modification date'],
                          parse_dates=['Modification date'])

In [75]:
#air_and_water.head()

In [76]:
air.head()

,WHO Country Name,City or Locality,Measurement Year,PM2.5 (μg/m3),PM10 (μg/m3),NO2 (μg/m3)
0,Afghanistan,Kabul,2019,119.77,NaN,NaN
1,Albania,Durres,2015,NaN,17.65,26.63
2,Albania,Durres,2016,14.32,24.56,24.78
3,Albania,Elbasan,2015,NaN,NaN,23.96
4,Albania,Elbasan,2016,NaN,NaN,26.26


In [77]:
development.head()

,country,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,hdi_1995,hdi_1996,hdi_1997,hdi_1998,...,hdi_2012,hdi_2013,hdi_2014,hdi_2015,hdi_2016,hdi_2017,hdi_2018,hdi_2019,hdi_2020,hdi_2021
0,Afghanistan,0.273,0.279,0.287,0.297,0.292,0.310,0.319,0.323,0.324,...,0.466,0.474,0.479,0.478,0.481,0.482,0.483,0.488,0.483,0.478
1,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.541,0.552,0.563,0.582,0.596,0.597,0.595,0.595,0.590,0.586
2,Albania,0.647,0.629,0.614,0.617,0.624,0.634,0.645,0.642,0.657,...,0.778,0.785,0.792,0.795,0.798,0.802,0.806,0.810,0.794,0.796
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.869,0.864,0.871,0.867,0.871,0.868,0.872,0.873,0.848,0.858
4,United Arab Emirates,0.728,0.739,0.742,0.748,0.755,0.762,0.767,0.773,0.779,...,0.846,0.852,0.859,0.865,0.870,0.897,0.909,0.920,0.912,0.911


In [78]:
gasses.head()

,country,year,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_including_luc,co2_including_luc_growth_abs,co2_including_luc_growth_prct,...,land_use_change_co2_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,oil_co2,oil_co2_per_capita,primary_energy_consumption,total_ghg,total_ghg_excluding_lucf
0,Afghanistan,1850,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,3767956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1852,3783940.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1853,3800954.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1854,3818038.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
gasses.shape

(50598, 45)

In [80]:
populations.head()

,Name,ASCII Name,Country name EN,Population,Modification date
0,Vehmersalmi,Vehmersalmi,Finland,1889,2014-09-25
1,Tornio,Tornio,Finland,21236,2014-09-25
2,Ruokolahti,Ruokolahti,Finland,5780,2014-09-25
3,Oitti,Oitti,Finland,0,2014-09-25
4,Noormarkku,Noormarkku,Finland,6076,2014-09-25


Finding what countries are in all dataframes

In [81]:
countries_air = air['WHO Country Name'].dropna().values
countries_dev = development['country'].dropna().values
countries_gas = gasses['country'].dropna().values
countries_pop = populations['Country name EN'].dropna().values

display(pd.unique(countries_air).shape)# It apears air has the least countries in it, but still 118 is prety good
display(pd.unique(countries_dev).shape)
display(pd.unique(countries_gas).shape)
display(pd.unique(countries_pop).shape)

(118,)

(206,)

(278,)

(229,)

In [82]:
common_countries = np.intersect1d(np.intersect1d(countries_air, countries_dev), np.intersect1d(countries_gas, countries_pop))
display(common_countries)
display(common_countries.shape)

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Argentina',
       'Australia', 'Austria', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Belgium', 'Bhutan', 'Bosnia and Herzegovina', 'Brazil',
       'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Denmark', 'Ecuador',
       'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Fiji', 'Finland',
       'France', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iraq',
       'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan',
       'Kazakhstan', 'Kenya', 'Kuwait', 'Kyrgyzstan', 'Latvia', 'Lebanon',
       'Liechtenstein', 'Lithuania', 'Luxembourg', 'Madagascar',
       'Malaysia', 'Maldives', 'Malta', 'Mauritius', 'Mexico', 'Mongolia',
       'Montenegro', 'Morocco', 'Myanmar', 'Nepal', 'Netherlands',
       'New Zealand', 'Nigeria', 'Norway', 'Pakistan', 'Panama',
       'Paraguay', 'Peru', 'P

(106,)

In [83]:
air.rename(columns={'WHO Country Name' : 'country', 'City or Locality' : 'city'}, inplace=True)
populations.rename(columns={'Country name EN' : 'country', 'Name' : 'city', 'ASCII Name' : 'ASCII city'}, inplace=True)

datasets = {
    'gasses' : gasses,
    'air' : air,
    'development' : development,
    'populations' : populations
}

In [84]:
for name, dataset in datasets.items():
  datasets[name] = dataset[dataset['country'].isin(common_countries)]
  datasets[name].reset_index(inplace=True, drop=True)

In [85]:
datasets['gasses'] = datasets['gasses'][datasets['gasses']['year'] >= 1990]

In [86]:
for name, dataset in datasets.items():
  display(dataset.shape)

(3392, 45)

(29197, 6)

(106, 33)

(110603, 5)

Finding the cities that are in both air and population dataframes

In [87]:
air_country_cities = datasets['air']['country'] + '_' + datasets['air']['city']
pop_country_cities = pd.concat([datasets['populations']['country'] + '_' + datasets['populations']['city'],
                                datasets['populations']['country'] + '_' + datasets['populations']['ASCII city']], ignore_index=True)


In [88]:
def sName(df):
  return df.str.lower().str.replace(' |-', '', regex=True)

In [89]:
common_idx = np.intersect1d(sName(air_country_cities), sName(pop_country_cities), return_indices=True)[1]
common_idx

array([    0,     1,     3, ..., 17357, 29048, 29050])

In [90]:
common_cities = datasets['air']['city'].iloc[common_idx]

datasets['air'] = datasets['air'][datasets['air']['city'].isin(common_cities)] #
datasets['air'].reset_index(inplace=True, drop=True)

datasets['populations'] = datasets['populations'][sName(datasets['populations']['city']).isin(sName(common_cities)) | sName(datasets['populations']['ASCII city']).isin(sName(common_cities))]
datasets['populations'].reset_index(inplace=True, drop=True)


In [91]:
datasets['air']

,country,city,Measurement Year,PM2.5 (μg/m3),PM10 (μg/m3),NO2 (μg/m3)
0,Afghanistan,Kabul,2019,119.77,NaN,NaN
1,Albania,Durres,2015,NaN,17.65,26.63
2,Albania,Durres,2016,14.32,24.56,24.78
3,Albania,Elbasan,2015,NaN,NaN,23.96
4,Albania,Elbasan,2016,NaN,NaN,26.26
...,...,...,...,...,...,...
20581,Uzbekistan,Tashkent,2019,41.96,NaN,NaN
20582,South Africa,Richards Bay,2012,13.00,NaN,NaN
20583,South Africa,Richards Bay,2017,NaN,28.41,NaN
20584,South Africa,Richards Bay,2018,NaN,28.37,NaN


In [92]:
datasets['air'].isna().sum()

country                 0
city                    0
Measurement Year        0
PM2.5 (μg/m3)       11805
PM10 (μg/m3)         5871
NO2 (μg/m3)          4558
dtype: int64

In [93]:
datasets['populations']

,city,ASCII city,country,Population,Modification date
0,Mikkeli,Mikkeli,Finland,52121,2022-08-13
1,Lucé,Luce,France,17780,2016-02-18
2,Laval,Laval,France,50489,2023-04-28
3,Ivry-sur-Seine,Ivry-sur-Seine,France,57897,2019-03-26
4,Istres,Istres,France,44373,2019-06-05
...,...,...,...,...,...
5033,Dumbrăveni,Dumbraveni,Romania,8110,2012-06-12
5034,Buenos Aires,Buenos Aires,Panama,496,2023-01-26
5035,Valencia,Valencia,Philippines,0,2017-12-13
5036,Bor,Bor,Serbia,39387,2019-09-05


In [94]:
datasets['gasses'].isna().sum()

country                                0
year                                   0
population                             0
gdp                                  494
cement_co2                             2
cement_co2_per_capita                  2
co2                                    0
co2_including_luc                     64
co2_including_luc_growth_abs          64
co2_including_luc_growth_prct         64
co2_including_luc_per_capita          64
co2_including_luc_per_gdp            494
co2_including_luc_per_unit_energy    136
co2_per_capita                         0
co2_per_gdp                          494
co2_per_unit_energy                  104
coal_co2                               0
coal_co2_per_capita                    0
cumulative_cement_co2                  2
cumulative_co2                         0
cumulative_co2_including_luc          64
cumulative_coal_co2                    0
cumulative_flaring_co2                 0
cumulative_gas_co2                     0
cumulative_luc_c